In [1]:
import QuantLib as ql

# Testing Hagan-pricer flat-vol equivalence for coupons...

In [2]:
termStructure = ql.RelinkableYieldTermStructureHandle()

In [3]:
calendar = ql.TARGET()

In [4]:
referenceDate = calendar.adjust(ql.Date.todaysDate())

In [5]:
ql.Settings.instance().evaluationDate = referenceDate

In [6]:
termStructure.linkTo(ql.FlatForward(referenceDate, 0.05,
                                          ql.Actual365Fixed()))

In [7]:
iborIndex = ql.Euribor6M(termStructure)

In [8]:
swapIndex = ql.SwapIndex("EuriborSwapIsdaFixA",
                         ql.Period("10Y"),
                          iborIndex.fixingDays(),
                          iborIndex.currency(),
                          iborIndex.fixingCalendar(),
                          ql.Period("1Y"),
                          ql.Unadjusted,
                          iborIndex.dayCounter(),
                          iborIndex)

In [9]:
startDate = termStructure.referenceDate() + 20*ql.Years

In [10]:
paymentDate = startDate + 1*ql.Years

In [11]:
endDate = paymentDate

In [12]:
nominal = 1.0
infiniteCap = ql.nullDouble()
infiniteFloor = ql.nullDouble()
gearing = 1.0
spread = 0.0

In [13]:
coupon = ql.CappedFlooredCmsCoupon(paymentDate, nominal,
                                  startDate, endDate,
                                  swapIndex.fixingDays(), swapIndex,
                                  gearing, spread,
                                  infiniteCap, infiniteFloor,
                                  startDate, endDate,
                                  iborIndex.dayCounter())

In [14]:
yieldCurveModels = [ql.GFunctionFactory.Standard,
                   ql.GFunctionFactory.ExactYield,
                   ql.GFunctionFactory.ParallelShifts,
                   ql.GFunctionFactory.NonParallelShifts,
                   ql.GFunctionFactory.NonParallelShifts]

In [15]:
zeroMeanRev = ql.QuoteHandle(ql.SimpleQuote(0.0))

In [16]:
atmOptionTenors = [
    ql.Period(1, ql.Months),
    ql.Period(6, ql.Months),
    ql.Period(1, ql.Years),
    ql.Period(5, ql.Years),
    ql.Period(10, ql.Years),
    ql.Period(30, ql.Years)

]

In [17]:
atmSwapTenors = [
    ql.Period(1, ql.Years),
    ql.Period(5, ql.Years),
    ql.Period(10, ql.Years),
    ql.Period(30, ql.Years)
]

In [18]:
m = ql.Matrix(len(atmOptionTenors),len(atmSwapTenors))

In [19]:
m[0][0]=0.1300; m[0][1]=0.1560; m[0][2]=0.1390; m[0][3]=0.1220;
m[1][0]=0.1440; m[1][1]=0.1580; m[1][2]=0.1460; m[1][3]=0.1260;
m[2][0]=0.1600; m[2][1]=0.1590; m[2][2]=0.1470; m[2][3]=0.1290;
m[3][0]=0.1640; m[3][1]=0.1470; m[3][2]=0.1370; m[3][3]=0.1220;
m[4][0]=0.1400; m[4][1]=0.1300; m[4][2]=0.1250; m[4][3]=0.1100;
m[5][0]=0.1130; m[5][1]=0.1090; m[5][2]=0.1070; m[5][3]=0.0930;

In [20]:
atmVol = ql.SwaptionVolatilityStructureHandle(
            ql.SwaptionVolatilityMatrix(calendar,
                                       ql.Following,
                                             atmOptionTenors,
                                             atmSwapTenors,
                                             m,
                                             ql.Actual365Fixed()))

In [21]:
numericalPricers = [ql.NumericHaganPricer(atmVol, yieldCurveModels[j], zeroMeanRev) for j in range(0, len(yieldCurveModels)-1)]
numericalPricers += [ql.LinearTsrPricer(atmVol, zeroMeanRev)]
analyticPricers = [ql.AnalyticHaganPricer(atmVol, yieldCurveModels[j],
                                        zeroMeanRev) for j in range(0, len(yieldCurveModels))]

In [22]:
for j in range(0, len(yieldCurveModels)):
    numericalPricers[j].setSwaptionVolatility(atmVol)
    coupon.setPricer(numericalPricers[j])
    rate0 = coupon.rate()
    analyticPricers[j].setSwaptionVolatility(atmVol)
    coupon.setPricer(analyticPricers[j])
    rate1 = coupon.rate()
    difference =  abs(rate1-rate0)
    print(difference)

3.873281508512405e-07
1.9250179429664005e-08
4.313882091822041e-09
4.313882091822041e-09
6.679072198090275e-08


# Testing Hagan-pricer flat-vol equivalence for swaps...

In [24]:
swapLengths = [1, 5, 6, 10]

In [141]:
# Should be SWIGed
def makeCMS(swapTenor : ql.Period,
            swapIndex : ql.SwapIndex,
            iborIndex : ql.IborIndex,
            iborSpread : float,
            forwardStart : ql.Period
           ):

    fixingDays = iborIndex.fixingDays()
    refDate = ql.Settings.instance().evaluationDate
    #if the evaluation date is not a business day
    #then move to the next business day
    floatCalendar = iborIndex.fixingCalendar()
    refDate = floatCalendar.adjust(refDate)
    spotDate = floatCalendar.advance(refDate, ql.Period(fixingDays, ql.Days))
    startDate = spotDate+forwardStart
    terminationDate = startDate+swapTenor

    cmsSchedule = ql.Schedule(startDate, terminationDate,
                              ql.Period(3, ql.Months), swapIndex.fixingCalendar(),
                              ql.ModifiedFollowing,
                              ql.ModifiedFollowing,
                              ql.DateGeneration.Backward, False,
                              ql.Date(), ql.Date())
    
    floatSchedule = ql.Schedule(startDate, terminationDate,
                                iborIndex.tenor(), floatCalendar,
                                iborIndex.businessDayConvention(),
                                iborIndex.businessDayConvention(),
                                ql.DateGeneration.Backward , False,
                                ql.Date(), ql.Date())

    cmsLeg = ql.CmsLeg([1.0], cmsSchedule, swapIndex, ql.Actual360(), ql.ModifiedFollowing, [swapIndex.fixingDays()], [1.0], [0.0], [], []
                       )

    floatLeg = ql.IborLeg([1.0], floatSchedule, iborIndex, iborIndex.dayCounter(), iborIndex.businessDayConvention(), [iborIndex.fixingDays()], [iborSpread])
    
    swap = ql.Swap(cmsLeg, floatLeg)
    engine = ql.DiscountingSwapEngine(swapIndex.forwardingTermStructure())
    swap.setPricingEngine(engine)
    return swap
    

In [142]:
cms = []

for swapLength in swapLengths:
    period = ql.Period(swapLength, ql.Years)
    
    swap = makeCMS(period, swapIndex, iborIndex, spread, ql.Period(10, ql.Days))

    cms+=[swap]

In [144]:
for j in range(0, len(yieldCurveModels)):
    numericalPricers[j].setSwaptionVolatility(atmVol)
    analyticPricers[j].setSwaptionVolatility(atmVol)

    for sl in range(0, len(cms)):
        #print(sl)
        ql.setCouponPricer(cms[sl].leg(0), numericalPricers[j])
        
        priceNum = cms[sl].NPV()
        ql.setCouponPricer(cms[sl].leg(0), analyticPricers[j])
        priceAn = cms[sl].NPV()

        difference =  abs(priceNum-priceAn)
        tol = 2.0e-4
        linearTsr = j==len(yieldCurveModels)-1

        print("Length in Years:  ", swapLengths[sl])
        #print("swap index:       " , swapIndex.name())
        #print("YieldCurve Model: " ,yieldCurveModels[j])
        print("Numerical Pricer: " ,(priceNum))
        print("Analytic Pricer:  " , (priceAn))
        print("difference:  ",difference)

Length in Years:   1
Numerical Pricer:  -0.04998591425676751
Analytic Pricer:   -0.0499868276912536
difference:   9.134344860911581e-07
Length in Years:   5
Numerical Pricer:  -0.22766453158670616
Analytic Pricer:   -0.22766845022437887
difference:   3.918637672711833e-06
Length in Years:   6
Numerical Pricer:  -0.2672632661492732
Analytic Pricer:   -0.2672629582820436
difference:   3.078672295986351e-07
Length in Years:   10
Numerical Pricer:  -0.40853925656533535
Analytic Pricer:   -0.40849447581580994
difference:   4.4780749525408226e-05
Length in Years:   1
Numerical Pricer:  -0.04998717213293465
Analytic Pricer:   -0.04998698248633201
difference:   1.8964660264408906e-07
Length in Years:   5
Numerical Pricer:  -0.22769665145567652
Analytic Pricer:   -0.22767226238973584
difference:   2.438906594068313e-05
Length in Years:   6
Numerical Pricer:  -0.26730762028624433
Analytic Pricer:   -0.2672682416204877
difference:   3.937866575665083e-05
Length in Years:   10
Numerical Pricer:  -